# Поправки
Для начала запустим sqlite и подключим нашу базу данных

In [1]:
%load_ext sql
%sql sqlite:///pizza_orders.db

Исправим неточности в нашей базе данных, напишем функцию которая преобразует datetime в нормальный вид

In [1]:
from datetime import datetime
import sqlite3

def translate(curr_time):

    unix_time = (curr_time - 2440587.5) * 86400
    dt = datetime.utcfromtimestamp(unix_time)
    dt_str = dt.strftime('%Y-%m-%d %H:%M:%S')

    return (dt.strftime('%Y-%m-%d %H:%M:%S')) 


with sqlite3.connect('pizza_orders.db') as db:
    cursor = db.cursor()
    db.create_function("translate", 1, translate)
    cursor.execute("UPDATE orders SET order_datetime = translate(order_datetime)")
    db.commit()

Убедимся, что теперь всё хорошо

In [7]:
%sql SELECT * FROM orders ORDER BY order_datetime LIMIT 5

 * sqlite:///pizza_orders.db
Done.


order_id,order_datetime
1,2015-01-01 11:38:36
2,2015-01-01 11:57:39
3,2015-01-01 12:12:28
4,2015-01-01 12:16:31
5,2015-01-01 12:21:30


# Задание 1
Найдите информацию по поводу ключевого слова HAVING запроса SELECT. Объясните, для чего оно нужно, приведите пример запроса с HAVING.

# Ответ: 
Having в SQL позволяет производить фильтрацию, после группировки данных(для чего WHERE совершенно не подходит)

In [14]:
%%sql
SELECT order_id, sum(quantity) as summary
FROM order_details
GROUP BY order_id
HAVING summary > 10
LIMIT 5


 * sqlite:///pizza_orders.db
Done.


order_id,summary
17,11
144,14
271,13
330,15
394,14


# Задание 2
Получите список всех имен таблиц и их запросов из таблицы sqlite_schema. 

In [11]:
%sql SELECT name, sql FROM sqlite_master WHERE type = 'table'


 * sqlite:///pizza_orders.db
Done.


name,sql
size,"CREATE TABLE size ( size_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, size_name TEXT NOT NULL )"
sqlite_sequence,"CREATE TABLE sqlite_sequence(name,seq)"
category,"CREATE TABLE category ( category_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, category_name TEXT NOT NULL )"
pizza,"CREATE TABLE pizza ( pizza_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, pizza_name TEXT NOT NULL, pizza_ingredients TEXT NOT NULL, category_id TEXT NOT NULL, FOREIGN KEY(category_id) REFERENCES category(category_id) )"
pizza_size,"CREATE TABLE pizza_size ( pizza_size_id TEXT NOT NULL PRIMARY KEY, pizza_id INTEGER NOT NULL, size_id INTEGER NOT NULL, unit_price REAL NOT NULL, FOREIGN KEY(pizza_id) REFERENCES pizza(pizza_id), FOREIGN KEY(size_id) REFERENCES size(size_id) )"
orders,"CREATE TABLE orders ( order_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, order_datetime DATETIME NOT NULL )"
order_details,"CREATE TABLE order_details ( order_details_id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT, order_id INTEGER NOT NULL, pizza_size_id TEXT NOT NULL, quantity INTEGER NOT NULL, FOREIGN KEY(order_id) REFERENCES orders(order_id), FOREIGN KEY(pizza_size_id) REFERENCES pizza_size(pizza_size_id) )"


# Задание 3
Получите перечень названий пицц и их ингредиентов.

In [16]:
%sql SELECT DISTINCT pizza_name as name, pizza_ingredients as ingredients FROM pizza

 * sqlite:///pizza_orders.db
Done.


name,ingredients
The Hawaiian Pizza,"Sliced Ham, Pineapple, Mozzarella Cheese"
The Classic Deluxe Pizza,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon"
The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic"
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic"
The Mexicana Pizza,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic"
The Thai Chicken Pizza,"Chicken, Pineapple, Tomatoes, Red Peppers, Thai Sweet Chilli Sauce"
The Prosciutto and Arugula Pizza,"Prosciutto di San Daniele, Arugula, Mozzarella Cheese"
The Barbecue Chicken Pizza,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
The Greek Pizza,"Kalamata Olives, Feta Cheese, Tomatoes, Garlic, Beef Chuck Roast, Red Onions"
The Spinach Supreme Pizza,"Spinach, Red Onions, Pepperoni, Tomatoes, Artichokes, Kalamata Olives, Garlic, Asiago Cheese"


# Задание 4
Покажите какие пиццы являются или вегетарианскими (“Veggie”) или куриными (“Chicken”). Также укажите перечень ингредиентов. 

In [20]:
%%sql
SELECT pizza_name, pizza_ingredients
FROM pizza pz
JOIN category ct ON pz.category_id = ct.category_id
WHERE ct.category_name IN('Chicken', 'Veggie')

 * sqlite:///pizza_orders.db
Done.


pizza_name,pizza_ingredients
The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic"
The Mexicana Pizza,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic"
The Thai Chicken Pizza,"Chicken, Pineapple, Tomatoes, Red Peppers, Thai Sweet Chilli Sauce"
The Barbecue Chicken Pizza,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
The Green Garden Pizza,"Spinach, Mushrooms, Tomatoes, Green Olives, Feta Cheese"
The Spinach Pesto Pizza,"Spinach, Artichokes, Tomatoes, Sun-dried Tomatoes, Garlic, Pesto Sauce"
The Vegetables + Vegetables Pizza,"Mushrooms, Tomatoes, Red Peppers, Green Peppers, Red Onions, Zucchini, Spinach, Garlic"
The Southwest Chicken Pizza,"Chicken, Tomatoes, Red Peppers, Red Onions, Jalapeno Peppers, Corn, Cilantro, Chipotle Sauce"
The California Chicken Pizza,"Chicken, Artichoke, Spinach, Garlic, Jalapeno Peppers, Fontina Cheese, Gouda Cheese"
The Chicken Pesto Pizza,"Chicken, Tomatoes, Red Peppers, Spinach, Garlic, Pesto Sauce"


# Задание 5
Какие пиццы содержат в себе моцареллу (“Mozzarella Cheese”). Также среди атрибутов укажите ингредиенты и названия категорий. 

In [22]:
%%sql
SELECT pizza_name, pizza_ingredients, ct.category_name
FROM pizza pz
JOIN category ct ON pz.category_id = ct.category_id
WHERE pizza_ingredients LIKE "%Mozzarella Cheese%"

 * sqlite:///pizza_orders.db
Done.


pizza_name,pizza_ingredients,category_name
The Hawaiian Pizza,"Sliced Ham, Pineapple, Mozzarella Cheese",Classic
The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic",Veggie
The Prosciutto and Arugula Pizza,"Prosciutto di San Daniele, Arugula, Mozzarella Cheese",Supreme
The Pepperoni Pizza,"Mozzarella Cheese, Pepperoni",Classic
The Soppressata Pizza,"Soppressata Salami, Fontina Cheese, Mozzarella Cheese, Mushrooms, Garlic",Supreme
The Four Cheese Pizza,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mozzarella Cheese, Parmigiano Reggiano Cheese, Garlic",Veggie


# Задание 6
Выведите полный список номеров заказов, которые были сделаны в промежутке с начала апреля 2015 года включительно по август 2015 года не включительно с 13:00 по 17:00. 

In [31]:
%%sql
WITH query as 
(
SELECT * FROM orders
WHERE DATE(order_datetime) BETWEEN '2015-04-01' AND '2015-07-31'
AND TIME(order_datetime) BETWEEN '13:00:00' AND '17:00:00'
)

SELECT count(*) FROM query

 * sqlite:///pizza_orders.db
Done.


count(*)
2531


In [32]:
%%sql
SELECT * FROM orders
WHERE DATE(order_datetime) BETWEEN '2015-04-01' AND '2015-07-31'
AND TIME(order_datetime) BETWEEN '13:00:00' AND '17:00:00'
LIMIT 5

 * sqlite:///pizza_orders.db
Done.


order_id,order_datetime
5388,2015-04-01 13:11:59
5389,2015-04-01 13:32:58
5390,2015-04-01 13:35:10
5391,2015-04-01 13:39:52
5392,2015-04-01 13:58:03


# Задание 7
Выведите номера заказов в порядке убывания количества купленных пицц. Ограничьте вывод 10-ю заказами. 

In [36]:
%sql SELECT order_id FROM order_details ORDER BY quantity DESC LIMIT 10

 * sqlite:///pizza_orders.db
Done.


order_id
5264
8286
15674
78
144
394
2612
3136
7404
8040


# Задание 8
Получите дату и время 10-ти заказов, в которых было куплено больше всего пицц. Упорядочьте записи по убыванию количества купленных позиций, по возрастанию даты и затем времени в лексикографическом порядке.

In [65]:
%%sql

SELECT ord.order_id, DATE(order_datetime) as date, TIME(order_datetime) as time, quantity 
FROM order_details ord_d
JOIN orders ord ON ord.order_id = ord_d.order_id
ORDER BY quantity DESC, date, time 
LIMIT 10

 * sqlite:///pizza_orders.db
Done.


order_id,date,time,quantity
5264,2015-03-30,13:24:34,4
8286,2015-05-19,12:42:46,4
15674,2015-09-19,14:16:35,4
78,2015-01-02,12:28:13,3
144,2015-01-03,13:44:27,3
394,2015-01-07,14:02:17,3
2612,2015-02-13,13:43:11,3
3136,2015-02-22,12:39:18,3
7404,2015-05-04,18:44:25,3
8040,2015-05-15,14:13:26,3


# Задание 9
Получите полное меню ресторана и цены на каждую позицию. В запросе должны быть отражены название, размер, стоимость, категория пиццы и ингредиенты, из которых она изготовлена. 

In [76]:
%%sql
SELECT pz.pizza_name, sz.size_name, pz_s.unit_price, ct.category_name, pz.pizza_ingredients
FROM pizza pz
JOIN pizza_size pz_s ON pz.pizza_id = pz_s.pizza_id
JOIN size sz ON pz_s.size_id = sz.size_id
JOIN category ct ON pz.category_id = ct.category_id


 * sqlite:///pizza_orders.db
Done.


pizza_name,size_name,unit_price,category_name,pizza_ingredients
The Hawaiian Pizza,M,13.25,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
The Classic Deluxe Pizza,M,16.0,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon"
The Five Cheese Pizza,L,18.5,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic"
The Italian Supreme Pizza,L,20.75,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic"
The Mexicana Pizza,M,16.0,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic"
The Thai Chicken Pizza,L,20.75,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Thai Sweet Chilli Sauce"
The Italian Supreme Pizza,M,16.5,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic"
The Prosciutto and Arugula Pizza,L,20.75,Supreme,"Prosciutto di San Daniele, Arugula, Mozzarella Cheese"
The Barbecue Chicken Pizza,S,12.75,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers, Tomatoes, Red Onions, Barbecue Sauce"
The Greek Pizza,S,12.0,Classic,"Kalamata Olives, Feta Cheese, Tomatoes, Garlic, Beef Chuck Roast, Red Onions"


# Задание 10
Выведите количество раз, когда каждая позиция меню (пицца и размер) была куплена не в единственном экземпляре (quantity != 1) за весь промежуток времени. 

In [89]:
%%sql 
SELECT pz.pizza_name,size_name as size, count() as count
FROM order_details ord_d 
JOIN pizza_size pz_s ON pz_s.pizza_size_id = ord_d.pizza_size_id
JOIN pizza pz ON pz.pizza_id = pz_s.pizza_id
JOIN size sz ON sz.size_id = pz_s.size_id 
WHERE quantity != 1
GROUP BY ord_d.pizza_size_id


 * sqlite:///pizza_orders.db
Done.


pizza_name,size,count
The Barbecue Chicken Pizza,L,24
The Barbecue Chicken Pizza,M,27
The Barbecue Chicken Pizza,S,5
The Big Meat Pizza,S,95
The Brie Carre Pizza,S,10
The Calabrese Pizza,L,2
The Calabrese Pizza,M,8
The California Chicken Pizza,L,30
The California Chicken Pizza,M,30
The California Chicken Pizza,S,6


# Задание 11
Получите полную таблицу транзакций и детализацию покупок за все время наблюдения (соединение всех таблиц в одну). Отдельными столбцами выведите дату покупки и время покупки, а также полную стоимость позиции, исходя из расчета на количество купленных товаров (total_price). 

In [5]:
%%sql
SELECT pz.pizza_name, pz.pizza_ingredients, ct.category_name, sz.size_name, pz_s.unit_price, ord_d.quantity, 
    DATE(order_datetime) as date, TIME(order_datetime) as time,(ord_d.quantity * pz_s.unit_price) as total_price
FROM pizza pz
JOIN category ct ON ct.category_id = pz.category_id
JOIN pizza_size pz_s ON pz_s.pizza_id = pz.pizza_id
JOIN size sz ON sz.size_id = pz_s.size_id
JOIN order_details ord_d ON ord_d.pizza_size_id = pz_s.pizza_size_id
JOIN orders ord ON ord.order_id = ord_d.order_id
LIMIT 25

 * sqlite:///pizza_orders.db
Done.


pizza_name,pizza_ingredients,category_name,size_name,unit_price,quantity,date,time,total_price
The Hawaiian Pizza,"Sliced Ham, Pineapple, Mozzarella Cheese",Classic,M,13.25,1,2015-01-01,11:38:36,13.25
The Classic Deluxe Pizza,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon",Classic,M,16.0,1,2015-01-01,11:57:39,16.0
The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic",Veggie,L,18.5,1,2015-01-01,11:57:39,18.5
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,L,20.75,1,2015-01-01,11:57:39,20.75
The Mexicana Pizza,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic",Veggie,M,16.0,1,2015-01-01,11:57:39,16.0
The Thai Chicken Pizza,"Chicken, Pineapple, Tomatoes, Red Peppers, Thai Sweet Chilli Sauce",Chicken,L,20.75,1,2015-01-01,11:57:39,20.75
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,M,16.5,1,2015-01-01,12:12:28,16.5
The Prosciutto and Arugula Pizza,"Prosciutto di San Daniele, Arugula, Mozzarella Cheese",Supreme,L,20.75,1,2015-01-01,12:12:28,20.75
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,M,16.5,1,2015-01-01,12:16:31,16.5
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,M,16.5,1,2015-01-01,12:21:30,16.5


# Задание 12
На полученную таблицу из задания 11 создайте представление с помощью команды CREATE VIEW "новое_имя_представления" AS ("ваш запрос"). 

In [14]:
%%sql

DROP VIEW IF EXISTS view_table;

CREATE VIEW view_table AS 
    SELECT pz.pizza_name, pz.pizza_ingredients, ct.category_name, sz.size_name, pz_s.unit_price, ord_d.quantity, 
    DATE(order_datetime) as date, TIME(order_datetime) as time,(ord_d.quantity * pz_s.unit_price) as total_price
    FROM pizza pz
    JOIN category ct ON ct.category_id = pz.category_id
    JOIN pizza_size pz_s ON pz_s.pizza_id = pz.pizza_id
    JOIN size sz ON sz.size_id = pz_s.size_id
    JOIN order_details ord_d ON ord_d.pizza_size_id = pz_s.pizza_size_id
    JOIN orders ord ON ord.order_id = ord_d.order_id



 * sqlite:///pizza_orders.db
Done.
Done.


[]

In [19]:
%sql SELECT * FROM view_table LIMIT 25

 * sqlite:///pizza_orders.db
Done.


pizza_name,pizza_ingredients,category_name,size_name,unit_price,quantity,date,time,total_price
The Hawaiian Pizza,"Sliced Ham, Pineapple, Mozzarella Cheese",Classic,M,13.25,1,2015-01-01,11:38:36,13.25
The Classic Deluxe Pizza,"Pepperoni, Mushrooms, Red Onions, Red Peppers, Bacon",Classic,M,16.0,1,2015-01-01,11:57:39,16.0
The Five Cheese Pizza,"Mozzarella Cheese, Provolone Cheese, Smoked Gouda Cheese, Romano Cheese, Blue Cheese, Garlic",Veggie,L,18.5,1,2015-01-01,11:57:39,18.5
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,L,20.75,1,2015-01-01,11:57:39,20.75
The Mexicana Pizza,"Tomatoes, Red Peppers, Jalapeno Peppers, Red Onions, Cilantro, Corn, Chipotle Sauce, Garlic",Veggie,M,16.0,1,2015-01-01,11:57:39,16.0
The Thai Chicken Pizza,"Chicken, Pineapple, Tomatoes, Red Peppers, Thai Sweet Chilli Sauce",Chicken,L,20.75,1,2015-01-01,11:57:39,20.75
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,M,16.5,1,2015-01-01,12:12:28,16.5
The Prosciutto and Arugula Pizza,"Prosciutto di San Daniele, Arugula, Mozzarella Cheese",Supreme,L,20.75,1,2015-01-01,12:12:28,20.75
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,M,16.5,1,2015-01-01,12:16:31,16.5
The Italian Supreme Pizza,"Calabrese Salami, Capocollo, Tomatoes, Red Onions, Green Olives, Garlic",Supreme,M,16.5,1,2015-01-01,12:21:30,16.5


# Задание 13
Подсчитать полный доход от разных категорий пицц за весь период наблюдения. Отсортируйте результат в порядке убывания дохода. Округлите результат подсчета дохода до второго знака после запятой.

In [24]:
%%sql
SELECT ROUND(SUM(ord_d.quantity * pz_s.unit_price), 2) as total, ct.category_name
FROM pizza pz
JOIN category ct ON ct.category_id = pz.category_id
JOIN pizza_size pz_s ON pz_s.pizza_id = pz.pizza_id
JOIN order_details ord_d ON ord_d.pizza_size_id = pz_s.pizza_size_id
GROUP BY ct.category_id
ORDER BY total DESC

 * sqlite:///pizza_orders.db
Done.


total,category_name
220053.1,Classic
208197.0,Supreme
195919.5,Chicken
193690.45,Veggie


# Задание 14
Пиццы какого размера продавались больше всего за 3-ий и за 4-ый квартал 2015 года? 

In [31]:
%%sql

WITH quarter_3 as
(
    SELECT count() as count,sz.size_name FROM
    orders ord 
    JOIN order_details ord_d ON ord_d.order_id = ord.order_id
    JOIN pizza_size pz_s ON ord_d.pizza_size_id = pz_s.pizza_size_id
    JOIN size sz ON sz.size_id = pz_s.size_id
    WHERE DATE(ord.order_datetime) BETWEEN '2015-07-01' AND '2015-09-30' 
    GROUP BY sz.size_id
    ORDER BY count DESC
    LIMIT 1
),

quarter_4 as
(
    SELECT count() as count, sz.size_name FROM
    orders ord 
    JOIN order_details ord_d ON ord_d.order_id = ord.order_id
    JOIN pizza_size pz_s ON ord_d.pizza_size_id = pz_s.pizza_size_id
    JOIN size sz ON sz.size_id = pz_s.size_id
    WHERE DATE(ord.order_datetime) BETWEEN '2015-10-01' AND '2015-12-31' 
    GROUP BY sz.size_id
    ORDER BY count DESC
    LIMIT 1
)

SELECT * FROM quarter_3 UNION SELECT * FROM quarter_4

 * sqlite:///pizza_orders.db
Done.


count,size_name
4502,L
4630,L
